In [18]:
import pandas as pd

In [60]:
games = pd.read_parquet('data_clean/1-games.parquet')
reviews = pd.read_parquet('data_clean/2-reviews.parquet')
items_spend = pd.read_parquet('data_clean/4-items_spend.parquet')
items_dev = pd.read_parquet('data_clean/7-items_dev.parquet')
usertime_year = pd.read_parquet('data_clean/8-usertime_year.parquet')
best_dev = pd.read_parquet('data_clean/9-best_dev.parquet')

In [20]:
items_dev.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28842 entries, 0 to 71042
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         28842 non-null  float64
 1   release_year  28842 non-null  int64  
 2   developer     28842 non-null  object 
 3   item_id       28842 non-null  int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 1.1+ MB


### Funcion *developer*

In [21]:
def developer(desarrollador:str):
    """
    Esta función devuelve la cantidad de items y el porcentaje de contenido gratuito por año para una empresa desarrolladora.

    Args:
        desarrollador (str): El nombre de la empresa desarrolladora.

    Returns:
        dict: Un diccionario con la cantidad de items y el porcentaje de contenido gratuito por año.
              Las claves son los años de lanzamiento y los valores son diccionarios con la siguiente estructura:
              {'cantidad_items': int, 'porcentaje_gratis': float}
    """
    # Filtrar el dataframe items_dev por la empresa desarrolladora especificada
    df_filtrado = items_dev[items_dev['developer'] == desarrollador]

    # Agrupar el dataframe filtrado por año de lanzamiento y calcular la cantidad de items y el conteo de contenido gratuito
    resultado = df_filtrado.groupby('release_year').agg(cantidad_items=('item_id', 'count'),
                                                        conteo_gratis=('price', lambda x: (x == 0.0).sum()))

    # Calcular el porcentaje de contenido gratuito por año
    resultado['porcentaje_gratis'] = (resultado['conteo_gratis'] / resultado['cantidad_items']) * 100

    return resultado.to_dict(orient='index')

In [38]:
desarrollador = 'Valve'
developer(desarrollador)

{1998: {'cantidad_items': 1, 'conteo_gratis': 0, 'porcentaje_gratis': 0.0},
 1999: {'cantidad_items': 1, 'conteo_gratis': 0, 'porcentaje_gratis': 0.0},
 2000: {'cantidad_items': 2, 'conteo_gratis': 0, 'porcentaje_gratis': 0.0},
 2001: {'cantidad_items': 1, 'conteo_gratis': 0, 'porcentaje_gratis': 0.0},
 2003: {'cantidad_items': 1, 'conteo_gratis': 0, 'porcentaje_gratis': 0.0},
 2004: {'cantidad_items': 5, 'conteo_gratis': 0, 'porcentaje_gratis': 0.0},
 2005: {'cantidad_items': 1, 'conteo_gratis': 1, 'porcentaje_gratis': 100.0},
 2006: {'cantidad_items': 2, 'conteo_gratis': 0, 'porcentaje_gratis': 0.0},
 2007: {'cantidad_items': 3,
  'conteo_gratis': 1,
  'porcentaje_gratis': 33.33333333333333},
 2008: {'cantidad_items': 1, 'conteo_gratis': 0, 'porcentaje_gratis': 0.0},
 2009: {'cantidad_items': 1, 'conteo_gratis': 0, 'porcentaje_gratis': 0.0},
 2010: {'cantidad_items': 2, 'conteo_gratis': 1, 'porcentaje_gratis': 50.0},
 2011: {'cantidad_items': 1, 'conteo_gratis': 0, 'porcentaje_gratis

### Función *userdata*

In [23]:
#imprimo info las tablas que voy a utilizar
items_spend.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68403 entries, 0 to 68402
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   items_count  68403 non-null  int64  
 1   user_id      68403 non-null  object 
 2   price        68403 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.6+ MB


In [24]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48690 entries, 0 to 48689
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             48690 non-null  object
 1   user_url            48690 non-null  object
 2   reviews_item_id     48690 non-null  int64 
 3   reviews_helpful     48690 non-null  object
 4   reviews_recommend   48690 non-null  bool  
 5   posted_year         48690 non-null  int64 
 6   sentiment_analysis  48690 non-null  int64 
 7   release_year        48690 non-null  int64 
dtypes: bool(1), int64(4), object(3)
memory usage: 2.6+ MB


In [25]:
def userData(user_id):
    """
    Esta función devuelve la cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews_recommend y cantidad de items.

    Parámetros:
    - user_id (int): Identificador único del usuario de interés.

    Devuelve:
    dict:
        - 'usuario_': Identificador único del usuario.
        - 'cantidad_dinero': Suma total de dinero gastado por el usuario en items_spend.
        - 'porcentaje_recomendacion': Porcentaje de recomendaciones realizadas por el usuario en comparación
          con el total de revisiones en el conjunto de datos.
        - 'total_items': La cantidad máxima de items comprados por el usuario en items_spend.
    """
    # Filtrar datos del usuario en items_spend
    usuario_gastos = items_spend[items_spend["user_id"] == user_id]

    # Calcular la cantidad de dinero gastado y el total de items para el usuario de interés
    cantidad_dinero = usuario_gastos["price"].sum()
    total_items = usuario_gastos["items_count"].max()

    # Calcular el total de recomendaciones realizadas por el usuario de interés
    total_recomendaciones = reviews[reviews["user_id"] == user_id]["reviews_recommend"].sum()

    # Calcular el total de revisiones realizadas por todos los usuarios
    total_reviews = len(reviews["user_id"].unique())

    # Calcular el porcentaje de recomendaciones realizadas por el usuario de interés
    porcentaje_recomendaciones = (total_recomendaciones / total_reviews) * 100

    # Retornar los resultados en un diccionario
    return {
        "usuario_": user_id,
        "cantidad_dinero": int(cantidad_dinero),
        "porcentaje_recomendacion": round(porcentaje_recomendaciones, 2),
        "total_items": int(total_items),
    }


In [39]:
user_id = 'doctr'
userData(user_id)

{'usuario_': 'doctr',
 'cantidad_dinero': 5125,
 'porcentaje_recomendacion': 0.03,
 'total_items': 541}

#### Función *UserForGenre*

In [27]:
usertime_year.info()

<class 'pandas.core.frame.DataFrame'>
Index: 67848 entries, 0 to 113499
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   playtime_forever  67848 non-null  float64
 1   user_id           67848 non-null  object 
 2   item_id           67848 non-null  int64  
 3   genres            67848 non-null  object 
 4   release_year      67848 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 3.1+ MB


In [28]:
def userForGenre(genero:str):
    # Filtrar el dataframe por el género dado
    usertime_year_filtrado = usertime_year[usertime_year['genres'] == genero]

    # Obtener el usuario con más horas jugadas para el género dado
    usuario_mas_jugado = usertime_year_filtrado.loc[usertime_year_filtrado['playtime_forever'].idxmax(), 'user_id']

    # Calcular la acumulación de horas jugadas por año de lanzamiento
    acumulacion_horas = usertime_year_filtrado.groupby('release_year')['playtime_forever'].sum().reset_index()
    acumulacion_horas = acumulacion_horas.rename(columns={'playtime_forever': 'Horas'})

    # Convertir el resultado a un diccionario
    resultado = {
        "Usuario con más horas jugadas para " + genero: usuario_mas_jugado,
        "Horas jugadas": acumulacion_horas.to_dict(orient='records')
    }

    return resultado

In [40]:
#ejemplo de uso
genero = 'Indie'
userForGenre(genero)

{'Usuario con más horas jugadas para Indie': '76561198055326819',
 'Horas jugadas': [{'release_year': 2004, 'Horas': 81.47},
  {'release_year': 2005, 'Horas': 77.37},
  {'release_year': 2006, 'Horas': 2363507.03},
  {'release_year': 2007, 'Horas': 27.39},
  {'release_year': 2008, 'Horas': 2145.02},
  {'release_year': 2009, 'Horas': 119.75},
  {'release_year': 2010, 'Horas': 1025.57},
  {'release_year': 2011, 'Horas': 215.25},
  {'release_year': 2012, 'Horas': 3112.83},
  {'release_year': 2013, 'Horas': 2256.46},
  {'release_year': 2014, 'Horas': 24.53},
  {'release_year': 2015, 'Horas': 1356.45},
  {'release_year': 2016, 'Horas': 120.47999999999999},
  {'release_year': 2017, 'Horas': 0.2}]}

### Función *best_developer_year*

In [30]:
best_dev.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6441 entries, 21 to 74446
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   price               6441 non-null   float64
 1   release_year        6441 non-null   int64  
 2   developer           6441 non-null   object 
 3   item_id             6441 non-null   int64  
 4   reviews_recommend   6441 non-null   bool   
 5   sentiment_analysis  6441 non-null   float64
dtypes: bool(1), float64(2), int64(2), object(1)
memory usage: 308.2+ KB


In [42]:
def best_developer_year(year):
    """
    Esta función devuelve el top 3 de desarrolladores con los juegos más recomendados y con un análisis de sentimiento igual a 2 para el año dado.

    Args:
        best_dev (pandas.DataFrame): El dataframe que contiene los datos.
        year (int): El año para el cual se desea obtener el top.

    Returns:
        list: Una lista con los nombres de los desarrolladores en el top 3.
    """
    # Filtrar el dataframe por el año y las condiciones de recomendación y análisis de sentimiento
    filtered_df = best_dev[(best_dev['release_year'] == year) & (best_dev['reviews_recommend'] == True) & (best_dev['sentiment_analysis'] == 2)]

    # Agrupar por desarrollador y contar el número de juegos recomendados
    developer_counts = filtered_df.groupby('developer').size().reset_index(name='count')

    # Ordenar en orden descendente y obtener los 3 primeros desarrolladores
    top_developers = developer_counts.nlargest(3, 'count')['developer'].tolist()
    
    #imprimir el resultado como un ranking del 1 al 3
    for i, developer in enumerate(top_developers):
        print(f'{i + 1}. {developer}')

    return top_developers

In [43]:
# Ejemplo de uso
year = 2010
best_developer_year(year)


1. BioWare
2. Obsidian Entertainment
3. Telltale Games


['BioWare', 'Obsidian Entertainment', 'Telltale Games']

### Función *developer_reviews_analysis*

In [61]:
def developer_reviews_analysis(desarrollador):
    """
    Analiza las reseñas de usuarios para un desarrollador específico y devuelve un diccionario con la cantidad total
    de registros de reseñas categorizados con un análisis de sentimiento como positivo o negativo.

    Parámetros:
    - desarrollador (str): Nombre del desarrollador para el cual se desea realizar el análisis.

    Retorna:
    dict: Un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de
    reseñas categorizados con un análisis de sentimiento como valor positivo o negativo.
    """
    # Filtra el DataFrame por el desarrollador deseado
    df_desarrollador = best_dev[best_dev['developer'] == desarrollador]

    # Cuenta la cantidad total de registros de reseñas categorizados como positivos o negativos
    sentiment_counts = df_desarrollador['sentiment_analysis'].value_counts().to_dict()

    # Asegurar que ambos sentimientos (positivo y negativo) están en el diccionario, incluso si son 0
    result = {
        desarrollador: {
            'Negativas': sentiment_counts.get(0, 0), 
            'Positivas': sentiment_counts.get(2, 0)
        }
    }

    return result


In [63]:
# Ejemplo de uso
desarrollador = 'Valve'
developer_reviews_analysis(desarrollador)

{'Valve': {'Negativas': 30, 'Positivas': 35}}